In [0]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

columns= ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data = simpleData, schema = columns)

display(df)

In [0]:
df.groupBy("department").agg({"salary": "sum"}).display()

In [0]:
df.groupBy("department").agg({"salary": "min"}).display()

In [0]:
df.groupBy("department").agg({"salary": "max"}).display()

In [0]:
df.groupBy("department").agg({"salary": "avg"}).display()

In [0]:
# groupby the department wit the sum of the salary and show all the employees name in the department, only employee name, department and salary, and sum of salary.
# df.groupBy("department").agg({"salary": "sum"}).join(df, "department").display()
df.groupBy("department").agg({"salary": "sum"}).join(df, "department").select("employee_name", "department", "salary").display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

windowSpec = Window.partitionBy('department')
ranked_df = df.withColumn('sum_slary', sum('salary').over(windowSpec))
ranked_df.show()

In [0]:
ranked_df.select("employee_name", "department", "salary", "sum_slary").display()

In [0]:
from pyspark.sql.functions import sum, avg, max

ranked_df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
         avg("salary").alias("avg_salary"), \
         sum("bonus").alias("sum_bonus"), \
         max("bonus").alias("max_bonus") \
     ) \
    .show(truncate=False)

In [0]:
from pyspark.sql.functions import col

ranked_df.where(col("age") > 30).display()
ranked_df.filter(col("age") > 30).display()

In [0]:
windowSpec = Window.partitionBy('state')
ranked_df2 = df.withColumn('sum_slary', sum('salary').over(windowSpec))
ranked_df2.display()
ranked_df2.filter(col("salary") > 90000).display()

In [0]:
path = "/FileStore/tables/testdata_sunil.csv"

test_df = spark.read.csv(path, header=True, inferSchema=True)
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
test_df = test_df.withColumnRenamed("5.1", "sepal_length").withColumnRenamed("3.5", "sepal_width").withColumnRenamed("1.4", "petal_length").withColumnRenamed("0.2", "petal_width").withColumnRenamed("Iris-setosa", "class")

test_df.display()

In [0]:
# in test_df get the records with the highest petal length and sepal lenght
a=test_df.agg(max("petal_length").alias("max petal")).collect() [0][0]
b=test_df.agg(max("sepal_length").alias("max sepal")).collect()[0][0]
print("max petal="+str(a))
print("max sepal="+str(b))

if a > b:
  print("petal length is greater than sepal length")
else:
  print("sepal length is greater than petal length")

In [0]:
test_df.filter(col("petal_length") == test_df.agg({'petal_length': 'max'}).collect()[0][0]).display()


In [0]:

simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

columns= ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data = simpleData, schema = columns)

display(df)

In [0]:
# cast a age integer to string
df.withColumn("age", df.age.cast("string")).display()

In [0]:
# multiply the salary by 10
df.withColumn("salary", df.salary * 10).display()

In [0]:
# Sample data
dataDF = [
    (("James", "", "Smith"), "36636", "M", 3100),
    (("Michael", "Rose", ""), "40288", "M", 4300),
    (("Robert", "", "Williams"), "42114", "M", 1400),
    (("Maria", "Anne", "Jones"), "39192", "F", 5500),
    (("Jen", "Mary", "Brown"), "", "F", -1)
]

from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([
    StructField('name', StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
        StructField('lastname', StringType(), True)
    ])),
    StructField('id', StringType(), True),
    StructField('gender', StringType(), True),
    StructField('salary', IntegerType(), True)
])

df = spark.createDataFrame(data=dataDF, schema=schema)
df.printSchema()
df.show(truncate=False)

In [0]:
df.withColumnRenamed("id", "new_id").withColumnRenamed("salary", "new_salary").display()

In [0]:
# rename the nested name cloumn elements
schema2 = StructType([
    StructField("f1name",StringType()),
    StructField("m1iddlename",StringType()),
    StructField("l1name",StringType())])
 
df.select(col("name").cast(schema2)).printSchema() 

In [0]:
data = [("James", "Sales", 3000), \
    ("Michael", "Sales", 4600), \
    ("Robert", "Sales", 4100), \
    ("Maria", "Finance", 3000), \
    ("James", "Sales", 3000), \
    ("Scott", "Finance", 3300), \
    ("Jen", "Finance", 3900), \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000), \
    ("Saif", "Sales", 4100) \
  ]
 
# Create DataFrame
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

In [0]:
# from pyspark.sql.functions import concat, lit
# create a new column with employee name and department as a array in the new cloumn

from pyspark.sql.functions import array

df = df.withColumn("emp_dep", array(df.employee_name, df.department))
# df.withColumn("emp_dep", concat(df.employee_name, lit("_"), df.department)).display()
df.display()

In [0]:
# show only disticnt records
df.select("employee_name", "department").distinct().display()

In [0]:
# remove duplicate record
df.dropDuplicates(["employee_name", "department"]).display()


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, MapType

temp_schema = StructType([StructField("address", MapType(StringType(), StringType()))])

data = [
    ("James", "Sales", 3000, {"address": {"city": "New York", "zip": "10001"}}),
    ("Michael", "Sales", 4600, {"address": {"city": "Chicago", "zip": "60601"}}),
    ("Robert", "Sales", 4100, {"address": {"city": "New York", "zip": "10001"}}),
    ("Maria", "Finance", 3000, {"address": {"city": "Seattle", "zip": "98101"}}),
    ("James", "Sales", 3000, {"address": {"city": "New York", "zip": "10001"}}),
    ("Scott", "Finance", 3300, {"address": {"city": "Seattle", "zip": "98101"}}),
    ("Jen", "Finance", 3900, {"address": {"city": "Chicago", "zip": "60601"}}),
    ("Jeff", "Marketing", 3000, {"address": {"city": "Chicago", "zip": "60601"}}),
    ("Kumar", "Marketing", 2000, {"address": {"city": "Seattle", "zip": "98101"}}),
    ("Saif", "Sales", 4100, {"address": {"city": "New York", "zip": "10001"}})
]

schema = StructType([
    StructField('name', StringType(), True),
    StructField('department', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('properties', temp_schema, True)
])

df = spark.createDataFrame(data = data, schema = schema)
df.printSchema()
df.show(truncate=False)

In [0]:
# Calculate minimum, maximum, average, and total salary per department.
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, min, max, avg

windowSpec = Window.partitionBy('department')
df2 = df.withColumn('min_salary', min('salary').over(windowSpec)) \
    .withColumn('max_salary', max('salary').over(windowSpec)) \
    .withColumn('avg_salary', avg('salary').over(windowSpec)) \
    .withColumn('total_salary', sum('salary').over(windowSpec))
df2.display()

In [0]:
# Filter out employees who earn more than 3000 and are in the Sales department.
df.filter((df.department == "Sales") & (df.salary > 3000)).display()

In [0]:
# Retrieve the maximum salary in the company using collect()[0][0].
df.select(max("salary")).collect()[0][0]

In [0]:
#If-Else Logic:
# Create a new column called "salary_level":
# "Low" if salary < 3000
# "Medium" if salary between 3000–4000
# "High" if salary > 4000

from pyspark.sql.functions import when

df3 = df.withColumn("salary_level", when(df.salary < 3000, "Low")
    .when((df.salary >= 3000) & (df.salary <= 4000), "Medium")    
    .otherwise("High"))

df3.display()

In [0]:
# withColumnRenamed:
# Rename "employee_name" to "name".

df4 = df.withColumnRenamed("name", "employee_name")
df4.display()

In [0]:
# Update Nested Schema:
# Update nested "address.city" to uppercase using withField.

from pyspark.sql.functions import col, upper, from_json
from pyspark.sql.types import StructType, StructField, StringType

# Define the schema for the properties column
properties_schema = StructType([
    StructField("address", StructType([
        StructField("city", StringType(), True)
    ]), True)
])

# Parse the properties column as JSON
df = df.withColumn("properties", from_json(col("properties"), properties_schema))

# Update the nested "address.city" field to uppercase
df5 = df.withColumn("properties", 
    col("properties").withField("address.city", 
        upper(col("properties.address.city"))))

display(df5)

In [0]:
# Remove duplicate rows (exact matches).
df.dropDuplicates().display()

In [0]:
# Assign a dense rank of salary within each department using Window.partitionBy.
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

windowSpec = Window.partitionBy('department').orderBy('salary')
ranked_df = df.withColumn('rank', dense_rank().over(windowSpec))
ranked_df.display()

In [0]:
from pyspark.sql.functions import col, upper, struct

df6 = df.withColumn(
    "properties",
    struct(
        struct(
            upper(col("properties.address.city")).alias("City"),
            col("properties.address.zip").alias("Zip")
        ).alias("Address")
    )
)

display(df6)